We conjecture that there exists a stochastic matrix $F(f) \neq P^*(f)$ for any finite MDP such that:
$$
P^*(f) = \lim_{n \to \infty} P^n(f)F(f).
$$ 

We call this the stripping property. (F for forgetting) The intuition is that $F(f)$ removes(strips) just enough information from $P^n(f)$ to make 
it converge to the cesaro limit. $F(f)$ isn't unique because the stripping property is preserved with multiplication 
with $P(f)$ or matrix multiplication. If a cheap stripping matrix exists then approximating $P^*(f)$ is cheap, because $P^{n}(f)$ can 
be calculated cheaply with square and multiply or as operator through unbiased sparsification. We  conjecture that:
$$
F(f) = \frac{1}{l} \sum_{k=0}^{l-1} P^{k}(f)= \frac{1}{l}(I-P^{l})(I-P)^{-1}.
$$ 
with $l$ is  the least common multiple of the periods of irreducible communication classes of P(f) and if $(I-P)^{-1}$ exists.
In this problem $l=1$ because the only irreducible classes possibles are the holes, the goal and 
one that contain "wall turn" backs.

In [4]:
from scipy.sparse import dok_array,eye
from scipy.sparse.linalg import norm 
from plt_utils import *

def power(P,n):
   if n%2==0 and n!=0:
        Q = power(P,n/2)
        return Q@Q 
   return power(P,n-1)@P if n!=0 else eye(P.shape[0])

def power_convergence(P,max_iter,eps =10**(-12)):
    Q=P
    for _ in range(max_iter):
        Qnew = Q@Q
        Qnew1 = P@Qnew
        if norm(Q-Qnew)<eps and norm(Qnew-Qnew1):break
        Q = Qnew1
    return Q 

def stripping_matrix(P,lcm_periods):
    return sum(power(P,j) for j in range(lcm_periods))/lcm_periods

def stationary_matrix(P,lcm_periods,iter=15, eps = 10**(-10)):
    Pn = power_convergence(P,iter,eps)
    Pn.data[Pn.data < eps] = 0
    Pn.eliminate_zeros()
    return Pn@stripping_matrix(P,lcm_periods)  

def stationary_matrix2(P,iter):
    I = eye(P.shape[0])
    B = I
    BB = [B]
    for i in range(1,iter):
        B = (i*P@B + I)/(i+1)
        BB.append(B)
    return BB

def stationary_matrix3(P):
    return stationary_matrix(0.9*P+0.1*eye(P.shape[0]),1,15)

In [5]:
P1 = dok_array((9,9))
P1[0,1] = 0.3
P1[0,2]= 0.7
P1[1,3] =1
P1[2,3]=1
P1[3,4]=1
P1[4,0] =1
P1[5,0] = 0.25
P1[5,1] = 0.25
P1[5,7] = 0.25
P1[5,8] = 0.25
P1[6,5] = 0.5
P1[6,3] = 0.5
P1[7,8] = 1
P1[8,7] = 1
print(P1.toarray())

[[0.   0.3  0.7  0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.25 0.   0.   0.   0.   0.   0.25 0.25]
 [0.   0.   0.   0.5  0.   0.5  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   0.  ]]


In [6]:
B1 = stationary_matrix(P1,4,2)
B2 = stationary_matrix3(P1)
BB1 = stationary_matrix2(P1,1000)
intPP([B1, B2])
intPP(BB1)

interactive(children=(IntSlider(value=0, description='index', max=1), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=0, description='index', max=999), Output()), _dom_classes=('widget-inter…

In [11]:
intPP([power(0.2*P1+0.8*eye(P1.shape[0]),j) for j in range(100)])

interactive(children=(IntSlider(value=0, description='index', max=99), Output()), _dom_classes=('widget-intera…